In [ ]:
# CREATING DB
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()


c.execute("""CREATE TABLE IF NOT EXISTS articles (
            headline text,  
            URL text UNIQUE,
            date text,
            is_nyhed integer,
            type_article text,
            text_body text,
            countries text,
            KM_index integer, 
            language text,
            media text,
            processed integer
            )""")


conn.commit()
conn.close()



In [ ]:
# GET COLUMN NAMES

conn = sqlite3.connect('database.db')
c = conn.execute('select * from articles')

# Fetch and print column names
column_names = [description[0] for description in c.description]
print("Column Names:", column_names)

conn.close()

Column Names: ['headline', 'URL', 'date', 'is_nyhed', 'type_article', 'text_body', 'countries', 'KM_index', 'language', 'media', 'processed']


In [26]:
# GET 10 ITEMS
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.execute('SELECT headline, URL, is_nyhed, type_article, text_body FROM articles')
# WHERE media = "ARD"


rows = c.fetchmany(25)
for row in rows:
    print(row)

conn.close()

('Waffenruhe in Gaza hält: Diese Geiseln warten auf ihre Freilassung', 'https://www.tagesschau.de/ausland/asien/geiseln-hamas-uebersicht-100.html', 1, 'ausland', 'Angekettet und gequält: Die Berichte der zuletzt freigelassenen Hamas-Geiseln aus Gaza lassen erahnen, was die 20 noch lebenden Geiseln durchmachen. Unter ihnen sind auch deutsche Staatsbürger. Eine Übersicht. Im Rahmen der Vereinbarung über eine Waffenruhe im Gaza-Krieg sollen nun zügig alle von der Hamas verschleppten Geiseln freigelassen werden. US-Präsident Donald Trump rechnet damit, dass die Übergabe am Montag stattfinden soll. Insgesamt sind noch 48 Menschen in der Gewalt der Terrororganisation im Gazastreifen, von denen 20 noch am Leben sein sollen. Sie werden unter grausamen Bedingungen festgehalten: Freigelassene Geiseln berichteten von Folter, schweren Misshandlungen, Hunger und schlimmen hygienischen Bedingungen. Wer sind die Männer, die am 7. Oktober 2023 beim Überfall der Terrororganisation Hamas auf Israel vers

In [24]:
# DELETE ALL DATA

import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('DELETE FROM articles')

# Commit the changes
conn.commit()
conn.close()

In [9]:
#  NUMBER OF ITEMS

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('SELECT * FROM articles')
print(len(c.fetchall()))

# Commit the changes
conn.commit()
conn.close()

16
